In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
text = open('shakespeare.txt','r').read()

In [3]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [4]:
vocab = sorted(list(set(text)))

In [5]:
stoi = {v:i for i,v in enumerate(vocab)}

In [6]:
itos = {v:k for k,v in stoi.items()}

In [7]:
vocab_size = len(stoi)

In [8]:
batch_size = 51
sequences = [text[i:i+batch_size]for i in range(0,len(text),batch_size)]

In [9]:
X = []
y = []
for seq in sequences:
    for i in range(1,len(seq)):
        enc = [stoi[c] for c in seq]
        X.append(enc[:i])
        y.append(enc[i])

In [10]:
maxlen = max([len(s) for s in X])

In [11]:
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X, padding='pre',maxlen=maxlen)
y_enc = tf.keras.utils.to_categorical(y, num_classes=len(stoi))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
...     X_padded, y_enc, test_size=0.33, random_state=42)

In [13]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=vocab_size,output_dim=256))
model.add(tf.keras.layers.LSTM(512))
# model.add(tf.keras.layers.Dense(32,activation='relu'))
model.add(tf.keras.layers.Dense(vocab_size,activation='softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         16640     
                                                                 
 lstm (LSTM)                 (None, 512)               1574912   
                                                                 
 dense (Dense)               (None, 65)                33345     
                                                                 
Total params: 1,624,897
Trainable params: 1,624,897
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=3e-4),loss='categorical_crossentropy')

In [30]:
model.fit(X_padded,y_enc,epochs=10,batch_size=256)

Epoch 1/10
4272/4272 [==============================] - 263s 61ms/step - loss: 1.0274
Epoch 2/10
4272/4272 [==============================] - 258s 60ms/step - loss: 0.9742
Epoch 3/10
4272/4272 [==============================] - 249s 58ms/step - loss: 0.9440
Epoch 4/10
4272/4272 [==============================] - 260s 61ms/step - loss: 0.9206
Epoch 5/10
4272/4272 [==============================] - 260s 61ms/step - loss: 0.9009
Epoch 6/10
4272/4272 [==============================] - 260s 61ms/step - loss: 0.8835
Epoch 7/10
4272/4272 [==============================] - 260s 61ms/step - loss: 0.8683
Epoch 8/10
4272/4272 [==============================] - 255s 60ms/step - loss: 0.8541
Epoch 9/10
4272/4272 [==============================] - 255s 60ms/step - loss: 0.8409
Epoch 10/10
4272/4272 [==============================] - 260s 61ms/step - loss: 0.8297


In [35]:
context = 'ROMEO:'
for i in range(1000):
    context_seq  = [[stoi[c] for c in context]]
    context_seq = tf.keras.preprocessing.sequence.pad_sequences(context_seq,maxlen=maxlen,padding='pre',truncating='pre')
    out = model.predict(context_seq,verbose=None);
    out /= 0.02
    pred_id = tf.random.categorical(out,num_samples=1)[0][0].numpy()
    context += itos[pred_id]

In [36]:
print(context)

ROMEO:
Is there no more shall be so banish'd with hers.

KING RICHARD III:
O Buckingham, I have some speech and down!
Is not the maid of curse,
When they do importune to her and understand.

KING EDWARD IV:
Now, brother of Gloucester, which way before
Being prison, defend the state to him that way to
come to accept the earth, and there they are now.
What, will he come?

COMINIUS:
What is the matter?

DUKE VINCENTIO:
He is a soul of the Grumio.

JULIET:
O this black day is but a little part
That thou hast slander'd with the street; therefore,
I am a poor brave form of the people,
And then to be her mother to the public body
To experies him dead; and therefore hath been
The state that do consume the field.

KING RICHARD III:
Say that I have still'd unto you to the world.

KING RICHARD III:
Why, then I was a man to the death or life,
And many a man of the sea within the water:
The rest, which is my father will make me behold
That thought to thee.

CAMILLO:
Sir, I pray you and I were not t

In [37]:
model.save('model.h5')